# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# Introduction
In this assignment, we will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information..

Installing the required module

In [1]:
!pip install bs4
!pip install geopy


Importing the required libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests  # this module helps us to download a web page
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
from sklearn.cluster import KMeans
from matplotlib import cm
import matplotlib.colors as colors


## Part 1

We will use the `read_html` function to directly get DataFrames from a `url` which defines the url of the website containg the table

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#html_data  = requests.get(url).text

In [4]:
toronto_data = pd.read_html(url, flavor='bs4')[0]

Retreiving the number of columns and rows in the dataset, we can see that there are 3 columns and 180 row

In [5]:
toronto_data.shape

(180, 3)

Displaying the first 5 rows of the dataset

In [6]:
toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Removing rows with a borough that is Not assigned.

In [7]:
toronto_data = toronto_data.loc[(toronto_data["Borough"] != 'Not assigned')]

Retreiving the number of columns and rows in the dataset, we can see that there are 3 columns and 103 rows after removing rows with a borough that is Not assigned.

In [8]:
toronto_data.shape

(103, 3)

The following command will check if a postal code is listed twice and has two neighborhoods, these two rows will be combined into one row with the neighborhoods separated with a comma.

There are no duplicate postalcodes in the dataset

In [9]:
toronto_data = toronto_data.groupby(['Postal Code'], as_index=False, sort=False).agg( ','.join)


In [10]:
toronto_data.shape

(103, 3)

Checking if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

There are no such cells with these criteria.

In [11]:
toronto_data.loc[toronto_data['Neighbourhood']=='Not assigned', 'Neighbourhood'] = toronto_data['Borough']

In [12]:
toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## part 2
Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name.

Now, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood which are stored in a csv file (obtained from https://cocl.us/Geospatial_data that has the geographical coordinates of each postal code in Toronto.

The following code will read the csv file that has the geographical coordinates of each postal code.

In [13]:
coord_data = pd.read_csv('https://cocl.us/Geospatial_data')
coord_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Joining the two dataframes together using the postal code as akey to result in the following final dataframe that has 5 colukmn names (Postal Code,Borough,Neighbourhood,Latitude,Longitude)

In [14]:
neighborhoods = toronto_data.join(coord_data.set_index('Postal Code'), on='Postal Code')
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


We make sure that the dataset has all 10 borough and 103 neighborhoods.

In [15]:
print('The dataframe has {} boroughs and {} neighbourhood.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0])
    )

The dataframe has 10 boroughs and 103 neighbourhood.


## Part 3

Use geopy library to get the latitude and longitude values of Toronto.


In [16]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


#### Create a map of Toronto with all neighborhoods superimposed on top.

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Explore and cluster the neighborhoods in Toronto

First, we will simplify the above map and segment and cluster only the neighborhoods with only boroughs that contain the word Toronto.

So let's slice the original dataframe.


In [18]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(neighborhoods.shape)
neighborhoods.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### Create a map of Toronto with neighborhoods with only boroughs that contain the word Toronto. superimposed on top.

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Start utilizing the Foursquare API to explore the neighborhoods and segment them.

Define Foursquare Credentials and Version

In [20]:
CLIENT_ID = 'G2I2IVNZR1LHDYZHDZPMYRAYN3LJPBNG2N40P1O2SI24UEAG' # your Foursquare ID
CLIENT_SECRET = 'BPWXAEL0J0ZTKOFDTTBZV2KXSNTZVEDK11VDEUMS5QOW1PM1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G2I2IVNZR1LHDYZHDZPMYRAYN3LJPBNG2N40P1O2SI24UEAG
CLIENT_SECRET:BPWXAEL0J0ZTKOFDTTBZV2KXSNTZVEDK11VDEUMS5QOW1PM1


Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [21]:
neighborhoods.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [22]:
# The following function retrieves the venues given the names and coordinates and stores it into dataframe.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(860, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Let's check the size of the resulting dataframe

In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,15,15,15,15,15,15
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


#### Let's find out how many unique categories can be curated from all the returned venues


In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 189 uniques categories.


## 3. Analyze Each Neighborhood


In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [28]:
toronto_onehot.shape

(860, 189)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [29]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's confirm the new size


In [30]:
toronto_grouped.shape

(39, 189)

#### Let's print each neighborhood along with the top 5 most common venues


In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1      Farmers Market  0.07
2         Coffee Shop  0.07
3        Cocktail Bar  0.07
4  Seafood Restaurant  0.07


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1  Breakfast Spot  0.08
2       Nightclub  0.08
3     Coffee Shop  0.08
4     Yoga Studio  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2                  Park  0.06
3           Pizza Place  0.06
4  Gym / Fitness Center  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4       Coffee Shop  0.07


----Central Bay Street----
                venue  freq
0       

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Bistro,Fish Market,Museum,Creperie,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Bar,Restaurant,Burrito Place,Climbing Gym,Yoga Studio
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Smoke Shop,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Rental Car Location,Harbor / Marina,Coffee Shop,Boutique,Airport Food Court
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Bubble Tea Shop,Poke Place,Modern European Restaurant


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
toronto_merged = toronto_venues

In [37]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio


Finally, let's visualize the resulting clusters


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.654260,-79.362017,Bakery,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
1,43.654260,-79.361809,Coffee Shop,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
2,43.654260,-79.358008,Distribution Center,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
3,43.654260,-79.359874,Spa,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
4,43.654260,-79.356980,Restaurant,0,Coffee Shop,Park,Theater,Bakery,Breakfast Spot,Café,Restaurant,Pub,Chocolate Shop,Yoga Studio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,43.662744,-79.324335,Yoga Studio,0,Yoga Studio,Smoke Shop,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Butcher
856,43.662744,-79.324768,Butcher,0,Yoga Studio,Smoke Shop,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Butcher
857,43.662744,-79.325379,Smoke Shop,0,Yoga Studio,Smoke Shop,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Butcher
858,43.662744,-79.326911,Gym / Fitness Center,0,Yoga Studio,Smoke Shop,Garden,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Pizza Place,Butcher


#### Cluster 2


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
657,43.689574,-79.383438,Park,1,Park,Trail,Restaurant,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
658,43.689574,-79.386841,Trail,1,Park,Trail,Restaurant,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
659,43.689574,-79.384504,Restaurant,1,Park,Trail,Restaurant,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
719,43.679563,-79.378934,Playground,1,Park,Playground,Trail,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
720,43.679563,-79.373788,Park,1,Park,Playground,Trail,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
721,43.679563,-79.382773,Park,1,Park,Playground,Trail,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
722,43.679563,-79.373842,Trail,1,Park,Playground,Trail,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


#### Cluster 3


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
185,43.669542,-79.420485,Grocery Store,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
186,43.669542,-79.426105,Café,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
187,43.669542,-79.426148,Italian Restaurant,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
188,43.669542,-79.419938,Café,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
189,43.669542,-79.421400,Coffee Shop,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
190,43.669542,-79.428054,Restaurant,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
191,43.669542,-79.421289,Candy Store,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
192,43.669542,-79.421364,Grocery Store,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
193,43.669542,-79.419297,Café,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant
194,43.669542,-79.427573,Grocery Store,2,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Coffee Shop,Nightclub,Cuban Restaurant


#### Cluster 4


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
470,43.72802,-79.394382,Park,3,Park,Swim School,Bus Line,College Cafeteria,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
471,43.72802,-79.382860,Swim School,3,Park,Swim School,Bus Line,College Cafeteria,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
472,43.72802,-79.382805,Bus Line,3,Park,Swim School,Bus Line,College Cafeteria,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


#### Cluster 5


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
473,43.711695,-79.415810,Health & Beauty Service,4,Garden,Health & Beauty Service,Home Service,Wine Bar,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
474,43.711695,-79.420702,Home Service,4,Garden,Health & Beauty Service,Home Service,Wine Bar,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
475,43.711695,-79.411978,Garden,4,Garden,Health & Beauty Service,Home Service,Wine Bar,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
